In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

**ANITA summer school on TDEs 2024**

Adelle Goodwin

**Part one: Fitting radio synchrotron spectra of TDEs**

Define some observational data

In [ ]:
#VLA observation taken on 2021 Feb 27 of AT2020vwl
freq1 = np.array([1.26,1.78, 3,4.5, 5.51, 6.49, 7.51, 9, 11]) #GHz
flux1 = np.array([244,294, 412,500, 543, 576, 564, 517, 464]) #uJy
flux_u1 = np.array([46,33, 10,14, 16, 14, 13, 9, 10]) #uJy


#VLA observation taken on 2021 Aug 11 of AT2020vwl
freq2 = np.array([1.284,1.52,2.5,3.24,4.49,5.51,6.49,7.51,9,11,15.08]) #GHz
flux2= np.array([367, 366,457,468,410,284,266,207,186, 147, 115]) #uJy
flux_u2 = np.array([22, 70,37,30,34,23,23,20,8.6,8.6, 6]) #uJy

Plot the data

In [ ]:
f = plt.figure(figsize=(5,5))

plt.scatter(freq1, flux1, label='VLA 2021 Feb 27', color='purple')
plt.errorbar(freq1, flux1, yerr=flux_u1,fmt='.',color='purple')

plt.scatter(freq2, flux2, label='VLA 2021 Aug 11',color='pink')
plt.errorbar(freq2, flux2, yerr=flux_u2,fmt='.',color='pink')


plt.legend(loc=3)
# we always plot radio spectra on a log-log scale. Don't ask why. I don't know. 
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Flux density (uJy)')
plt.title('AT2020vwl VLA observations')

Let's do a phenomenological fit to the power-law indices either side of the breaks

In [ ]:
# define a powerlaw function
def powerlaw_simple(x,a,b):
    return a*x**b

In [ ]:
# fit the simple power law to epoch 1 data below the break, i.e. the first 6 data points

from scipy.optimize import curve_fit

popt, pcov = curve_fit(f=powerlaw_simple, xdata=freq1[:6], ydata=flux1[:6])

f = plt.figure(figsize=(5,5))

plt.plot(freq1[:6], powerlaw_simple(freq1[:6], *popt),label='simple fit')

plt.scatter(freq1, flux1, label='VLA 2021 Feb 27', color='purple')
plt.errorbar(freq1, flux1, yerr=flux_u1,fmt='.',color='purple')
plt.legend(loc=3)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Flux density (uJy)')

print(f'The power-law index below the break is {popt[1]}')

In [ ]:
# fit the simple power law to epoch 1 data above the break, i.e. the last 3 data points

popt, pcov = curve_fit(f=powerlaw_simple, xdata=freq1[-3:], ydata=flux1[-3:])

f = plt.figure(figsize=(5,5))

plt.plot(freq1[-3:], powerlaw_simple(freq1[-3:], *popt),label='simple fit')

plt.scatter(freq1, flux1, label='VLA 2021 Feb 27', color='purple')
plt.errorbar(freq1, flux1, yerr=flux_u1,fmt='.',color='purple')
plt.legend(loc=3)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Flux density (uJy)')

print(f'The power-law index above the break is {popt[1]}')

In [ ]:
# repeat for the August epoch

# fit the simple power law to epoch 2 data below the break, i.e. the first 4 data points

popt, pcov = curve_fit(f=powerlaw_simple, xdata=freq2[:4], ydata=flux2[:4])

f = plt.figure(figsize=(5,5))

plt.plot(freq2[:4], powerlaw_simple(freq2[:4], *popt),label='simple fit')

plt.scatter(freq2, flux2, label='VLA 2021 Aug 11', color='pink')
plt.errorbar(freq2, flux2, yerr=flux_u2,fmt='.',color='pink')
plt.legend(loc=3)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Flux density (uJy)')

print(f'The power-law index below the break is {popt[1]}')

In [ ]:
# repeat for the August epoch

# fit the simple power law to epoch 2 data above the break, i.e. the last 8 data points

popt, pcov = curve_fit(f=powerlaw_simple, xdata=freq2[-8:], ydata=flux2[-8:])

f = plt.figure(figsize=(5,5))

plt.plot(freq2[-8:], powerlaw_simple(freq2[-8:], *popt),label='simple fit')

plt.scatter(freq2, flux2, label='VLA 2021 Aug 11', color='pink')
plt.errorbar(freq2, flux2, yerr=flux_u2,fmt='.',color='pink')
plt.legend(loc=3)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Flux density (uJy)')


print(f'The power-law index below the break is {popt[1]}, i.e. p={1-2*popt[1]}')

In [ ]:
# Now let's try a physically motivated synchrotron emission model from Granot+Sari (2002) 
# for the synchrotron self-absorption break vm < vsa < vc

def GS02_powerlaw(v, Fvb, vb, va, p):

            beta2 = 5/2
            beta3 =(1-p)/2 
            s4 = 3.63*p - 1.60
            s2 = 1.25-0.18*p

            Fv1 = Fvb * ((v / vb) ** 2 * np.exp(-s4*(v/vb)**(2/3)) + \
                         (v / vb) ** (5/2)) * (1 + (v/va)**(s2*(beta2-beta3)))**(-1/s2)
            return Fv1

In [ ]:
popt, pcov = curve_fit(f=GS02_powerlaw, xdata=freq2, ydata=flux2, p0=(1e3,0.5,5,2.5), \
                       bounds=([1e-3,0.1,0.1,2],[1e6,15,15,3.5]))

f = plt.figure(figsize=(5,5))

plt.plot(freq2, GS02_powerlaw(freq2, *popt),label='GS02 fit')

plt.scatter(freq2, flux2, label='VLA 2021 Aug 11', color='pink')
plt.errorbar(freq2, flux2, yerr=flux_u2,fmt='.',color='pink')
plt.legend(loc=3)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Flux density (uJy)')


print(f'The synchrotron power-law index p is {popt[3]}')

In [ ]:
# do the fit
popt, pcov = curve_fit(f=GS02_powerlaw, xdata=freq1, ydata=flux1, p0=(1e3,0.5,5,2.5), \
                       bounds=([1e-3,0.1,0.1,2],[1e6,15,15,3.5]))

# plot the output
f = plt.figure(figsize=(5,5))

plt.plot(freq1, GS02_powerlaw(freq1, *popt),label='GS02 fit')

plt.scatter(freq1, flux1, label='VLA 2021 Feb 27', color='purple')
plt.errorbar(freq1, flux1, yerr=flux_u1,fmt='.',color='purple')
plt.legend(loc=3)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Flux density (uJy)')

# print the synchrotron powerlaw index
print(f'The synchrotron power-law index p is {popt[3]}')

The February epoch is not a good fit. 
This could be because the early time low-frequency emission is dominated by galaxy host emission.
Let's try subtracting a host component and then fit again

In [ ]:
# define host component, where the total flux is Ftot = Fsynch + Fhost
# see Goodwin+2023b for how the host spectrum was constrained
host = (0.178e3)*(freq1/1.4)**(-1.088)

# do the fit again, subtracting the host component this time
popt, pcov = curve_fit(f=GS02_powerlaw, xdata=freq1, ydata=flux1-host, p0=(1e3,0.5,5,2.5), \
                       bounds=([1e-3,0.1,0.1,2],[1e6,15,15,3.5]))

# plot the results

f = plt.figure(figsize=(5,5))

plt.plot(freq1, GS02_powerlaw(freq1, *popt),label='GS02 fit')

plt.scatter(freq1, flux1-host, label='VLA 2021 Feb 27 host sub', color='purple')
plt.errorbar(freq1, flux1-host, yerr=flux_u1,fmt='.',color='purple')
plt.legend(loc=3)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Flux density (uJy)')

# print the synchrotron power-law index
print(f'The synchrotron power-law index p is {popt[3]}')

This is a much better fit!

Now you can try fitting a different source on your own, e.g. AT2020opy from Goodwin+2023a

In [ ]:
# Define the observed quantitites
flux1 = np.array([53,60,75, 91,134,139, 133.8, 137, 136.7,98.8]) #2020 Dec 20
flux_u1 = np.array([13,13,11, 12, 15,11,8.3,9.45, 10,10])
freq1 = np.array([3.24,3.754,4.49, 5.51,6.5,7.5, 9.0, 11., 13.5,16.5])

flux2 = np.array([251,228,252,256,241,175, 198,139]) #2021 June 21
flux_u2 = np.array([100,27,17,17,16,14,9,11])
freq2 = np.array([2.5,3.5,4.5,5.5,6.49,7.5,9,11])

In [ ]:
# first plot the data

f = plt.figure(figsize=(5,5))




In [ ]:
# Now choose a model and fit the spectra


In [ ]:
# Plot the results

**Part two: extracting physical outflow properties from synchrotron spectral properties**

Based on the equipartition method outlined by Barniol Duran+2013 and your synchrotron constraints from above, determine the radius, energy, ambient denisty, velocity, and mass in the emitting region for each epoch of each source. 